## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
<br>
<center>ФБ-22 Дажук Павло</center>

#### Хід виконання

In [1]:
import os
import pandas as pd
pd.set_option('display.max_rows', None)
import urllib
from datetime import datetime
import re
print("Setup Complete")

Setup Complete


In [2]:
regions = {
    1: 'Vinnytsya',  2: 'Volyn', 3: 'Dnipropetrovsk', 4: 'Donetsk', 5: 'Zhytomyr',
    6: 'Zakarpattia', 7: 'Zaporizhia', 8: 'Ivano-Frankivsk', 9: 'Kyiv', 10: 'Kirovohrad',
    11: 'Luhansk', 12: 'Lviv', 13: 'Mykolaiv', 14: 'Odessa', 15: 'Poltava',
    16: 'Rivne', 17: 'Sumy', 18: 'Ternopil', 19: 'Kharkiv', 20: 'Kherson',
    21: 'Khmelnytsky', 22: 'Cherkasy', 23: 'Chernivtsi', 24: 'Chernihiv', 25: 'Crimea'
}

In [3]:
def download_csv():
    ids = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    
    for i in ids.keys():
        id = ids[i]
        folder_path = "data"
            
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        for file in os.listdir(folder_path):
            if file.startswith(f"obl_{id}_"):
                break
        else:
            url='https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2020&type=Mean'.format(i)
            wp = urllib.request.urlopen(url)
            text = wp.read()
            
            text = text.replace(b"<br>", b"")
            text = text.replace(b"<tt><pre>", b"")
            text = text.replace(b"</pre></tt>", b"")
            
            now = datetime.now()
            date_and_time_time = now.strftime("%d-%m-%Y_%H-%M-%S")
            name = f"data\\obl_{id}_{date_and_time_time}.csv"
            out=open(name, 'wb')
            out.write(text)
            out.close()
            print (f"{name}, ", end="")

In [4]:
download_csv()

In [5]:
def handling_values(folder_path):
    dfs = []
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Region_ID']
    
    file_list = os.listdir(folder_path)
    
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, header=1, names=headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        region_id = re.search(r'_(\d+)_', file_name).group(1)
        df['Region_ID'] = int(region_id)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [6]:
df = handling_values('data')
df.to_csv('df.csv')

o Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [7]:
def analyze_vhi_for_year(df, regions, region, year):
    vhi_series = df[(df['Year'] == year) & (df['Region_ID'] == region)][['VHI']]
    min_vhi = vhi_series.min().values[0]
    max_vhi = vhi_series.max().values[0]
    
    print(vhi_series, "\n")
    print(f"{regions[region]} in {year}:")
    print(f"Min VHI: {min_vhi}")
    print(f"Max VHI: {max_vhi}")

In [8]:
analyze_vhi_for_year(df, regions, 2, 2000)

         VHI
34532  24.65
34533  27.49
34534  31.36
34535  37.28
34536  40.85
34537  41.53
34538  41.41
34539  42.27
34540  44.28
34541  45.89
34542  47.46
34543  50.11
34544  50.60
34545  50.12
34546  49.23
34547  52.96
34548  59.41
34549  65.21
34550  68.54
34551  68.85
34552  65.40
34553  60.29
34554  55.39
34555  52.26
34556  51.63
34557  50.86
34558  51.05
34559  51.94
34560  55.96
34561  59.19
34562  59.25
34563  59.77
34564  59.68
34565  60.68
34566  63.06
34567  63.67
34568  62.06
34569  63.94
34570  61.73
34571  51.99
34572  47.37
34573  44.83
34574  42.03
34575  37.46
34576  35.60
34577  36.51
34578  38.84
34579  41.41
34580  43.38
34581  44.57
34582  47.99
34583  43.78 

Volyn in 2000:
Min VHI: 24.65
Max VHI: 68.85


o Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [9]:
def analyze_vhi_for_years_and_regions(df, regions, start_year, end_year, selected_regions):
    vhi_series = df[(df['Year'] >= start_year) & (df['Year'] <= end_year) & (df['Region_ID'].isin(selected_regions))][['Year', 'VHI', 'Region_ID']]
    print(vhi_series)


In [10]:
analyze_vhi_for_years_and_regions(df, regions, 2000, 2003, [1, 2, 3])

       Year    VHI  Region_ID
20686  2000  24.22          1
20687  2000  27.70          1
20688  2000  30.68          1
20689  2000  32.55          1
20690  2000  34.73          1
20691  2000  35.08          1
20692  2000  33.79          1
20693  2000  34.60          1
20694  2000  37.70          1
20695  2000  38.67          1
20696  2000  38.05          1
20697  2000  39.32          1
20698  2000  40.21          1
20699  2000  41.25          1
20700  2000  43.66          1
20701  2000  47.61          1
20702  2000  52.54          1
20703  2000  60.14          1
20704  2000  62.91          1
20705  2000  63.27          1
20706  2000  61.76          1
20707  2000  58.32          1
20708  2000  56.10          1
20709  2000  55.32          1
20710  2000  55.03          1
20711  2000  54.68          1
20712  2000  55.27          1
20713  2000  56.28          1
20714  2000  57.93          1
20715  2000  57.94          1
20716  2000  57.29          1
20717  2000  56.45          1
20718  200

In [11]:
# Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей
def find_extreme_drought_years(df, regions, percentage):
    total_regions = len(regions)
    affected_years = []

    for year in range(1982, 2021):
        affected_regions = 0

        for region in range(1, total_regions + 1):
            min_vhi = df[(df['Year'] == year) & (df['Region_ID'] == region)]['VHI'].min()

            if min_vhi < 15:
                affected_regions += 1

        if affected_regions/total_regions > percentage / 100: 
            affected_years.append(year)
    
    print(f"Years with extreme drought in more than {percentage}% of regions: {affected_years}")


In [12]:
find_extreme_drought_years(df, regions, 5)

Years with extreme drought in more than 5% of regions: [1986, 1993, 2000, 2007]


o Аналогічно для помірних посух

In [13]:
def find_moderate_drought_years(df, regions, percentage):
    total_regions = len(regions)
    affected_years = []

    for year in range(1982, 2021):
        affected_regions = 0

        for region in range(1, total_regions + 1):
            vhi = df[(df['Year'] == year) & (df['Region_ID'] == region) & (df['VHI'] >= 15)]['VHI'].min()

            if vhi < 35:
                affected_regions += 1

        if affected_regions/total_regions > percentage / 100: 
            affected_years.append(year)
    
    print(f"Years with moderate drought in more than {percentage}% of regions: {affected_years}")


In [14]:
find_moderate_drought_years(df, regions, 95)

Years with moderate drought in more than 95% of regions: [1982, 1983, 1984, 1985, 1986, 1989, 1995, 1996, 2000, 2019]
